In [4]:
from sympy import symbols 
w=symbols("w")

In [ ]:

# from src.acoustic_analyser import frame
# import numpy as np

# test_frame = frame.from_file(member_file="test_member.json", constraint_file="test_constraint.csv")

# free_end = test_frame.fixed_end(member_id=0)
# free_end = test_frame.free_end(member_id=1)

# matrix = test_frame.get_equation_matrix(w=3)


In [1]:

from src.acoustic_analyser import frame
import numpy as np

test_frame = frame()

test_frame.add_member(length=0.5, density=7800, youngs_modulus=206e9, height=1.27e-2,id=0)
test_frame.add_member(length=0.5, density=7800, youngs_modulus=206e9, height=1.27e-2,id=1)

free_end = test_frame.fixed_end(member_id=0)
joint = test_frame.two_member_joint(member_1_id=0,member_2_id=1, theta=45)
free_end = test_frame.free_end(member_id=1)

matrix = test_frame.get_equation_matrix(w=3)


2022-12-22 19:18:47,367 src.modules.rt_joint DEBUG:Equation File Loaded
2022-12-22 19:18:49,454 src.modules.member DEBUG:Propagation Matrix Calculated for member 0
2022-12-22 19:18:49,456 src.modules.member DEBUG:Parameters set for member 0
2022-12-22 19:18:49,458 src.acoustic_analyser DEBUG:Member added with id 0
2022-12-22 19:18:49,460 src.modules.member DEBUG:Propagation Matrix Calculated for member 1
2022-12-22 19:18:49,463 src.modules.member DEBUG:Parameters set for member 1
2022-12-22 19:18:49,466 src.acoustic_analyser DEBUG:Member added with id 1
2022-12-22 19:18:49,468 src.modules.bc DEBUG:Reflection Matrix for free_end 0 calculated
2022-12-22 19:18:49,469 src.acoustic_analyser DEBUG:Fixed End added to member 0 with id 0
2022-12-22 19:18:49,473 src.acoustic_analyser DEBUG:Two member joint added b/w 0 and 1 with id 1
2022-12-22 19:18:49,475 src.modules.bc DEBUG:Reflection Matrix for free_end 2 calculated
2022-12-22 19:18:49,476 src.acoustic_analyser DEBUG:Free End added to membe

In [5]:
test_frame.members[0].propagation_matrix.subs(w,3).evalf(4)

Matrix([
[0.9802 - 0.1982*I,      0,               0],
[                0, 0.8191,               0],
[                0,      0, 1.0 - 0.00029*I]])

In [6]:
abs(np.linalg.det(matrix))

0.2624467383843455

In [ ]:
# y = []
# x = np.arange(1,400,50)
# for freq in x:
#     w = 2*np.pi*freq
#     matrix = test_frame.get_equation_matrix(w=w)
#     y.append(abs(np.linalg.det(matrix)))
#     print(freq)

# from matplotlib import pyplot as plt
# plt.plot(x,y)